In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('datasets/missing_value_imputed.csv')

In [3]:
viz_df = pd.read_csv('datasets/Data Visualization 1.csv')

In [21]:
viz_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3329 entries, 0 to 3328
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   property_type    3329 non-null   object 
 1   society          3329 non-null   object 
 2   sector           3329 non-null   object 
 3   price            3329 non-null   float64
 4   price_per_sqft   3329 non-null   float64
 5   bedRoom          3329 non-null   float64
 6   bathroom         3329 non-null   float64
 7   balcony          3329 non-null   object 
 8   floorNum         3329 non-null   float64
 9   agePossession    3329 non-null   object 
 10  built_up_area    3329 non-null   float64
 11  study room       3329 non-null   float64
 12  servant room     3329 non-null   float64
 13  store room       3329 non-null   float64
 14  pooja room       3329 non-null   float64
 15  others           3329 non-null   float64
 16  furnishing_type  3329 non-null   float64
 17  luxury_score  

In [6]:
import plotly.express as px

In [16]:

px.set_mapbox_access_token(None)

fig = px.scatter_map(
    viz_df,
    lat="latitude",
    lon="longitude",
    color="price_per_sqft",
    size="built_up_area",
    color_continuous_scale=px.colors.cyclical.IceFire,
    zoom=10,
    map_style="carto-positron",  # modern free tile layer
    width=1200,
    height=700,
    hover_name=viz_df.index,
)

fig.update_layout(
    margin=dict(l=0, r=0, t=40, b=0),
    title="🏙️ Property Price per Sq Ft (Carto Map)",
)





ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': ('<b>%{hovertext}</b><br><br>bui' ... '%{marker.color}<extra></extra>'),
              'hovertext': {'bdata': ('AAAAAAAAAAAAAAAAAADwPwAAAAAAAA' ... 'AA/KlAAAAAAAD+qUAAAAAAAACqQA=='),
                            'dtype': 'f8'},
              'lat': {'bdata': ('0SLb+X5qPEDRItv5fmo8QNEi2/l+aj' ... 'EIbDxAEHo2qz53PEAQejarPnc8QA=='),
                      'dtype': 'f8'},
              'legendgroup': '',
              'lon': {'bdata': ('SFD8GHM/U0BIUPwYcz9TQEhQ/BhzP1' ... 'BBPVNAxEKtad5FU0DEQq1p3kVTQA=='),
                      'dtype': 'f8'},
              'marker': {'color': {'bdata': ('AAAAAACivUAAAAAAAEy9QAAAAACAVs' ... 'AAn7pAAAAAAEC+3kAAAAAAgK7ZQA=='),
                                   'dtype': 'f8'},
                         'coloraxis': 'coloraxis',
                         'size': {'bdata': ('AAAAAACQikAAAAAAAMCKQAAAAAAAiJ' ... 'AAGJVAAAAAAAAYlUAAAAAAAMSnQA=='),
                                  'dtype': 'f8'},
                         'sizemode': 'area',
                         'sizeref': np.float64(30.555)},
              'mode': 'markers',
              'name': '',
              'showlegend': False,
              'subplot': 'map',
              'type': 'scattermap'}],
    'layout': {'coloraxis': {'colorbar': {'title': {'text': 'price_per_sqft'}},
                             'colorscale': [[0.0, '#000000'], [0.0625, '#001f4d'],
                                            [0.125, '#003786'], [0.1875,
                                            '#0e58a8'], [0.25, '#217eb8'], [0.3125,
                                            '#30a4ca'], [0.375, '#54c8df'],
                                            [0.4375, '#9be4ef'], [0.5, '#e1e9d1'],
                                            [0.5625, '#f3d573'], [0.625,
                                            '#e7b000'], [0.6875, '#da8200'], [0.75,
                                            '#c65400'], [0.8125, '#ac2301'],
                                            [0.875, '#820000'], [0.9375,
                                            '#4c0000'], [1.0, '#000000']]},
               'height': 700,
               'legend': {'itemsizing': 'constant', 'tracegroupgap': 0},
               'map': {'center': {'lat': np.float64(28.436744638029438), 'lon': np.float64(77.00856984079303)},
                       'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
                       'style': 'carto-positron',
                       'zoom': 10},
               'mapbox': {'center': {'lat': np.float64(28.436744638029438), 'lon': np.float64(77.00856984079303)},
                          'style': 'carto-positron',
                          'zoom': 10},
               'margin': {'b': 0, 'l': 0, 'r': 0, 't': 40},
               'template': '...',
               'title': {'text': '🏙️ Property Price per Sq Ft (Carto Map)'},
               'width': 1200}
})

In [20]:
import h3
lat_col = "latitude"
lon_col = "longitude"
value_col = "price_per_sqft"
h3_res = 8   # try 7 (bigger cells) … 9 (smaller cells)
# Add H3 index per row
viz_df["h3_index"] = viz_df.apply(
    lambda r: h3.latlng_to_cell(r[lat_col], r[lon_col], h3_res),
    axis=1
)

# Aggregate (mean per hex)
agg_df = (
    viz_df.groupby("h3_index", as_index=False)[value_col]
          .mean()
          .rename(columns={value_col: "value"})
)
def h3_fc_from_indexes(indexes):
    """Return a GeoJSON FeatureCollection for given H3 cell indexes."""
    features = []
    for h in indexes:
        boundary = h3.cell_to_boundary(h, geo_json=True)  # list of [lat, lon]
        # Convert to [lon, lat] and close the ring
        coords = [[lng, lat] for lat, lng in boundary]
        if coords[0] != coords[-1]:
            coords.append(coords[0])

        feat = {
            "type": "Feature",
            "properties": {"h3_index": h},
            "geometry": {
                "type": "Polygon",
                "coordinates": [coords],
            },
        }
        features.append(feat)
    return {"type": "FeatureCollection", "features": features}

hex_geojson = h3_fc_from_indexes(agg_df["h3_index"])
center_lat = viz_df[lat_col].mean()
center_lon = viz_df[lon_col].mean()

fig = px.choropleth_mapbox(
    agg_df,
    geojson=hex_geojson,
    locations="h3_index",
    featureidkey="properties.h3_index",
    color="value",
    color_continuous_scale=px.colors.cyclical.IceFire,
    mapbox_style="carto-positron",   # zoomable tiles, no token required
    opacity=0.65,
    zoom=11,
    center={"lat": center_lat, "lon": center_lon},
    height=700,
)

fig.update_layout(
    title=f"Average {value_col} by H3 Hex (res={h3_res})",
    margin=dict(l=0, r=0, t=50, b=0),
    coloraxis_colorbar=dict(title=value_col),
)

fig.update_traces(
    hovertemplate="<b>Hex:</b> %{location}<br>"
                  f"<b>{value_col}:</b> %{z:.0f}<extra></extra>"
)

fig.show()


TypeError: cell_to_boundary() got an unexpected keyword argument 'geo_json'